In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt


In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset), shuffle=False)


In [3]:
train_data_iter = iter(trainloader)
train_images, train_labels = next(train_data_iter)

test_data_iter = iter(testloader)
test_images, test_labels = next(test_data_iter)

train_images = train_images.view(-1, 28*28).numpy()
train_labels = train_labels.numpy()

test_images = test_images.view(-1, 28*28).numpy()
test_labels = test_labels.numpy()

train_labels = train_labels.ravel()
test_labels = test_labels.ravel()



In [4]:
# plt.imshow(train_images[10].reshape(28, 28))

In [5]:
dtrain = xgb.DMatrix(train_images, label=train_labels)
dtest = xgb.DMatrix(test_images, label=test_labels)

param = {
    'max_depth': 6,
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 10,
    'eval_metric': 'mlogloss'
}

num_round = 100
evallist  = [(dtrain, 'eval'), (dtest, 'eval')]
bst = xgb.train(param, dtrain, num_round, evals=evallist, verbose_eval=True)

[0]	eval-mlogloss:1.96071
[1]	eval-mlogloss:1.72771
[2]	eval-mlogloss:1.54761
[3]	eval-mlogloss:1.40229
[4]	eval-mlogloss:1.28212
[5]	eval-mlogloss:1.17831
[6]	eval-mlogloss:1.08765
[7]	eval-mlogloss:1.00655
[8]	eval-mlogloss:0.93493
[9]	eval-mlogloss:0.87134
[10]	eval-mlogloss:0.81401
[11]	eval-mlogloss:0.76197
[12]	eval-mlogloss:0.71554
[13]	eval-mlogloss:0.67219
[14]	eval-mlogloss:0.63363
[15]	eval-mlogloss:0.59769
[16]	eval-mlogloss:0.56497
[17]	eval-mlogloss:0.53528
[18]	eval-mlogloss:0.50787
[19]	eval-mlogloss:0.48283
[20]	eval-mlogloss:0.45918
[21]	eval-mlogloss:0.43776
[22]	eval-mlogloss:0.41739
[23]	eval-mlogloss:0.39922
[24]	eval-mlogloss:0.38167
[25]	eval-mlogloss:0.36643
[26]	eval-mlogloss:0.35166
[27]	eval-mlogloss:0.33806
[28]	eval-mlogloss:0.32522
[29]	eval-mlogloss:0.31329
[30]	eval-mlogloss:0.30227
[31]	eval-mlogloss:0.29163
[32]	eval-mlogloss:0.28203
[33]	eval-mlogloss:0.27297
[34]	eval-mlogloss:0.26453
[35]	eval-mlogloss:0.25639
[36]	eval-mlogloss:0.24877
[37]	eval-m

In [7]:
preds = bst.predict(dtest)

accuracy = accuracy_score(test_labels, preds)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 97.02%


In [8]:
preds = bst.predict(dtrain)

accuracy = accuracy_score(train_labels, preds)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 99.60%
